DATE : 05/09/2020

CLASSWORK 

HADOOP STREAMING

INSTALL HADOOP

In [ ]:
!wget http://apachemirror.wuchna.com/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

--2020-09-05 09:12:06--  http://apachemirror.wuchna.com/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
Resolving apachemirror.wuchna.com (apachemirror.wuchna.com)... 159.65.154.237
Connecting to apachemirror.wuchna.com (apachemirror.wuchna.com)|159.65.154.237|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500749234 (478M) [application/x-gzip]
Saving to: ‘hadoop-3.3.0.tar.gz.2’

hadoop-3.3.0.tar.gz 100%[===================>] 477.55M  33.1MB/s    in 15s     

2020-09-05 09:12:21 (32.2 MB/s) - ‘hadoop-3.3.0.tar.gz.2’ saved [500749234/500749234]



In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Streaming output truncated to the last 5000 lines.
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/TrashPolicyDefault.Emptier.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/HarFileSystem.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/PathExistsException.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/XAttrSetFlag.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/BatchedRemoteIterator.BatchedEntries.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-comm

In [ ]:
!sudo mv hadoop-3.3.0 /usr/local/hadoop

In [ ]:
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/



edit the .sh file manually

export JAVA_HOME=$(readlink -f /usr/bin/java | sed "s:bin/java::")


In [ ]:
!/usr/local/hadoop/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python
import fileinput,re,io
from contextlib import redirect_stdout
import os
import nltk,sys
from nltk.corpus import stopwords
with redirect_stdout(open(os.devnull, "w")): #suppress the output
    nltk.download('stopwords')
sw = stopwords.words('english') 
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer('english')

def remove_stopwords(text):
  filtered_words = [w for w in text.split()  if ((not w in sw) and (len(w)>1))]
  return " ".join(filtered_words)
stem = lambda x:snowball.stem(x)
def preprocess():
  lines = []
  input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding='latin1')
  for line in input_stream:
    line = line.strip()                           #Strip the whitespaces
    line = line.replace("\n", " ")                #Replace newlines with ''
    line = line.lower()                           #convert into lowercase
    line = re.sub("[^a-zA-Z]", " ",line)          #keep only text
    line = remove_stopwords(line)                 #remove stopwords
    line = ' '.join(list(map(stem,line.split()))) #stemming
    lines.append(line)
  return lines
preprocessed = preprocess()
f = lambda x:(x,1)
new_list = []
for line in preprocessed:
  line = line.split()
  new_list.extend(list(map(f,line)))
import itertools
chain = itertools.chain(new_list)
for x,y in list(chain):
  print(x,"\t",y)

Overwriting mapper.py


In [ ]:
!chmod u+rwx /content/mapper.py

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python
import fileinput,sys,io
new_word = None
new_count = 0
input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding='latin1')
for line in input_stream:
  current_word, current_count = line.split()
  if current_word == new_word:
    new_count += int(current_count)
    continue
  else:
    if new_word:
      print(new_word,new_count)
    new_word = current_word
    new_count = 1
#for the last word
print(new_word,new_count)


Overwriting reducer.py


In [ ]:
!chmod u+rwx /content/reducer.py

INSTALL DATA

In [ ]:
!wget --no-check-certificate http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz -O /content/20news-bydate.tar.gz

--2020-09-05 11:45:03--  http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/x-gzip]
Saving to: ‘/content/20news-bydate.tar.gz’

/content/20news-byd 100%[===================>]  13.79M  3.77MB/s    in 3.7s    

2020-09-05 11:45:07 (3.77 MB/s) - ‘/content/20news-bydate.tar.gz’ saved [14464277/14464277]



In [ ]:
!tar -xvzf /content/20news-bydate.tar.gz -C /content

Streaming output truncated to the last 5000 lines.
20news-bydate-train/rec.sport.hockey/53814
20news-bydate-train/rec.sport.hockey/53815
20news-bydate-train/rec.sport.hockey/53878
20news-bydate-train/rec.sport.hockey/53821
20news-bydate-train/rec.sport.hockey/53811
20news-bydate-train/rec.sport.hockey/53818
20news-bydate-train/rec.sport.hockey/53812
20news-bydate-train/rec.sport.hockey/53813
20news-bydate-train/rec.sport.hockey/53823
20news-bydate-train/rec.sport.hockey/53819
20news-bydate-train/rec.sport.hockey/53817
20news-bydate-train/rec.sport.hockey/54094
20news-bydate-train/rec.sport.hockey/53824
20news-bydate-train/rec.sport.hockey/53891
20news-bydate-train/rec.sport.hockey/53829
20news-bydate-train/rec.sport.hockey/53899
20news-bydate-train/rec.sport.hockey/53830
20news-bydate-train/rec.sport.hockey/53847
20news-bydate-train/rec.sport.hockey/53831
20news-bydate-train/rec.sport.hockey/53833
20news-bydate-train/rec.sport.hockey/53909
20news-bydate-train/rec.sport.hockey/53910
20n

In [ ]:
!iconv -f 'latin1' /content/20news-bydate-train/*/* > mergedfiles

In [ ]:
!cat mergedfiles | head -n 20

From: mathew <mathew@mantis.co.uk>
Subject: Alt.Atheism FAQ: Atheist Resources
Summary: Books, addresses, music -- anything related to atheism
Keywords: FAQ, atheism, books, music, fiction, addresses, contacts
Expires: Thu, 29 Apr 1993 11:57:19 GMT
Distribution: world
Organization: Mantis Consultants, Cambridge. UK.
Supersedes: <19930301143317@mantis.co.uk>
Lines: 290

Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modified: 11 December 1992
Version: 1.0

                              Atheist Resources

                      Addresses of Atheist Organizations

                                     USA


In [ ]:
!find / -name 'hadoop-streaming*.jar'

/usr/local/hadoop/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
/usr/local/hadoop/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-sources.jar
/usr/local/hadoop/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-test-sources.jar
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
/usr/local/hadoop/share/hadoop/tools/sources/hadoop-streaming-3.3.0-sources.jar
/usr/local/hadoop/share/hadoop/tools/sources/hadoop-streaming-3.3.0-test-sources.jar


In [ ]:
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -D stream.jobconf.truncate.limit=20000 \
-input /content/mergedfiles \
-output /content/output \
-mapper /content/mapper.py \
-reducer /content/reducer.py

2020-09-05 11:45:16,104 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-09-05 11:45:16,254 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-09-05 11:45:16,254 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-09-05 11:45:16,271 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-09-05 11:45:16,449 INFO mapred.FileInputFormat: Total input files to process : 1
2020-09-05 11:45:16,469 INFO mapreduce.JobSubmitter: number of splits:1
2020-09-05 11:45:16,740 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1487944728_0001
2020-09-05 11:45:16,740 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-09-05 11:45:17,017 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2020-09-05 11:45:17,019 INFO mapreduce.Job: Running job: job_local1487944728_0001
2020-09-05 11:45:17,030 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2020-09

In [ ]:
!cat output/part-00000 | tail -n 20

zznkj 1	
zznkjrlb 1	
zznkjz 1	
zznkzz 1	
zznm 1	
zznowu 1	
zznp 1	
zzo 1	
zzpgvz 1	
zzpn 1	
zzr 2	
zzrk 1	
zzt 1	
zztop 1	
zzum 2	
zzz 1	
zzzoh 1	
zzzz 2	
zzzzzz 4	
zzzzzzt 1	


In [ ]:
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -D stream.jobconf.truncate.limit=20000 \
-input /content/20news-bydate-train/*/* \
-output /content/output1 \
-mapper /content/mapper.py \
-reducer /content/reducer.py

Streaming output truncated to the last 5000 lines.
2020-09-05 13:00:26,214 INFO mapred.LocalJobRunner: 
2020-09-05 13:00:26,214 INFO mapred.MapTask: Starting flush of map output
2020-09-05 13:00:26,214 INFO mapred.MapTask: Spilling map output
2020-09-05 13:00:26,214 INFO mapred.MapTask: bufstart = 0; bufend = 1538; bufvoid = 104857600
2020-09-05 13:00:26,214 INFO mapred.MapTask: kvstart = 26214396(104857584); kvend = 26213792(104855168); length = 605/6553600
2020-09-05 13:00:26,215 INFO mapred.MapTask: Finished spill 0
2020-09-05 13:00:26,216 INFO mapred.Task: Task:attempt_local1698488060_0001_m_003017_0 is done. And is in the process of committing
2020-09-05 13:00:26,216 INFO mapred.LocalJobRunner: Records R/W=39/1
2020-09-05 13:00:26,217 INFO mapred.Task: Task 'attempt_local1698488060_0001_m_003017_0' done.
2020-09-05 13:00:26,217 INFO mapred.Task: Final Counters for attempt_local1698488060_0001_m_003017_0: Counters: 17
	File System Counters
		FILE: Number of bytes read=39084881
		FI

In [ ]:
!cat output1/part-00000 | tail -n 20